<a href="https://colab.research.google.com/github/PacoDLC/deeplearning-az/blob/master/rbm.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Máquinas de Boltzmann


## Clonar repositorio de GitHub para acceder a los datasets

In [19]:
!git clone https://github.com/PacoDLC/deeplearning-az

fatal: destination path 'deeplearning-az' already exists and is not an empty directory.


## Montar conexión a Google Drive

In [20]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


#Importación de las librerías

In [21]:
import numpy as np
import pandas as pd
# Librerías de PyTorch
import torch
import torch.nn as nn # Neural Network
import torch.nn.parallel # Cálculos en paralelo (optimización)
import torch.optim as optim # Optimizador para minimizar el error
import torch.utils.data
from torch.autograd import Variable

#Importación del conjunto de datos

In [22]:
movies = pd.read_csv("/content/deeplearning-az/updated/Part 5 - Boltzmann Machines (BM)/ml-1m/movies.dat",
                     sep = '::', header = None, engine = 'python', encoding = 'latin-1')
users  = pd.read_csv("/content/deeplearning-az/updated/Part 5 - Boltzmann Machines (BM)/ml-1m/users.dat",
                     sep = '::', header = None, engine = 'python', encoding = 'latin-1')
ratings  = pd.read_csv("/content/deeplearning-az/updated/Part 5 - Boltzmann Machines (BM)/ml-1m/ratings.dat",
                       sep = '::', header = None, engine = 'python', encoding = 'latin-1')

Estructura del DataFrame `movies`

* Columna `0`: identificador de la película,
* Columna `1`: nombre de la película (año de estreno),
* Columna `2`: categoría de la película.

In [23]:
movies.head()

,0,1,2
0,1,Toy Story (1995),Animation|Children's|Comedy
1,2,Jumanji (1995),Adventure|Children's|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama
4,5,Father of the Bride Part II (1995),Comedy


Estructura del DataFrame `users`

* Columna `0`: identificador del usuario,
* Columna `1`: género del ususario (F - mujer, M - hombre),
* Columna `2`: edad del usuario,
* Columna `3`: empleo u ocupación del usuario (codificado),
* Columns `4`: código postal del usuario.

In [24]:
users.head()

,0,1,2,3,4
0,1,F,1,10,48067
1,2,M,56,16,70072
2,3,M,25,15,55117
3,4,M,45,7,02460
4,5,M,25,20,55455


Estructura del DataFrame `ratings`

* Columna `0`: identificador del usuario,
* Columna `1`: identificador de la película que vió el usuario,
* Columna `2`: valoración que el usuario otorga a la película,
* Columna `3`: timestamp (irrelevante en este caso).

In [25]:
ratings.head()

,0,1,2,3
0,1,1193,5,978300760
1,1,661,3,978302109
2,1,914,3,978301968
3,1,3408,4,978300275
4,1,2355,5,978824291


# Preparación del conjunto de entrenamiento y el conjunto de pruebas


La estructura de los DataFrames de entrenamiento (`training_set_DF`) y prueba (`test_set_DF`) es la misma que la del DataFrame `ratings`. Estos conjuntos deben ser transformados en arreglos matriciales de NumPy y deben estar conformados por un sólo tipo de datos (en este caso elegimos un tipo de dato entero `int`) para poder procesarlos por la Red Neuronal.

In [26]:
training_set_DF = pd.read_csv("/content/deeplearning-az/updated/Part 5 - Boltzmann Machines (BM)/ml-100k/u1.base",
                           sep = "\t", header = None)
training_set = np.array(training_set_DF, dtype = "int")
test_set_DF = pd.read_csv("/content/deeplearning-az/updated/Part 5 - Boltzmann Machines (BM)/ml-100k/u1.test",
                       sep = "\t", header = None)
test_set = np.array(test_set_DF, dtype = "int")

In [27]:
training_set_DF.head()

,0,1,2,3
0,1,1,5,874965758
1,1,2,3,876893171
2,1,3,4,878542960
3,1,4,3,876893119
4,1,5,3,889751712


In [28]:
test_set_DF.head()

,0,1,2,3
0,1,6,5,887431973
1,1,10,3,875693118
2,1,12,5,878542960
3,1,14,5,874965706
4,1,17,3,875073198


# Obtención del número de usuarios y películas


In [29]:
nb_users = int(max(max(training_set[:,0]), max(test_set[:,0])))
nb_movies = int(max(max(training_set[:,1]), max(test_set[:,1])))
print(f"Número de usuarios:  {nb_users}")
print(f"Número de películas: {nb_movies}")

Número de usuarios:  943
Número de películas: 1682


# Conversión de los datos en una matriz con los usuarios en líneas y las películas en columnas

En esta parte se creará una estructura de datos de tal manera que a cada usuario se le asignará una fila, del mismo modo, a cada película se le asignará una columna en esta estructura. De este modo, se creará una lista de listas en dónde cada lista dentro de esta lista contendrá las valoraciones que el usuario de ha dado a la película, en caso de no haber visto la película se colocará un `0`.

In [30]:
def convert(data):
    new_data = []
    for id_users in range(1, nb_users + 1):
        id_movies = data[:,1][data[:,0] == id_users]
        id_ratings = data[:,2][data[:,0] == id_users]
        ratings = np.zeros(nb_movies)
        ratings[id_movies - 1] = id_ratings
        new_data.append(list(ratings))
    return new_data
training_set = convert(training_set)
test_set = convert(test_set)

# Conversión de los datos en Torch tensors

In [31]:
training_set = torch.FloatTensor(training_set)
test_set = torch.FloatTensor(test_set)

# Convertir las clasificaciones en clasificaciones binarias 1 (Le gusta) o 0 (No le gusta)

In [32]:
training_set[training_set == 0] = -1
training_set[training_set == 1] = 0
training_set[training_set == 2] = 0
training_set[training_set >= 3] = 1
test_set[test_set == 0] = -1
test_set[test_set == 1] = 0
test_set[test_set == 2] = 0
test_set[test_set >= 3] = 1

# Creación de la arquitectura de la red neuronal (modelo probabilístico gráfico)

In [ ]:
class RBM():

    def __init__(self, nv, nh):
        self.W = torch.randn(nh, nv)
        self.a = torch.randn(1, nh)
        self.b = torch.randn(1, nv)

    def sample_h(self, x):
        wx = torch.mm(x, self.W.t())
        activation = wx + self.a.expand_as(wx)
        p_h_given_v = torch.sigmoid(activation)
        return p_h_given_v, torch.bernoulli(p_h_given_v)

    def sample_v(self, y):
        wy = torch.mm(y, self.W)
        activation = wy + self.b.expand_as(wy)
        p_v_given_h = torch.sigmoid(activation)
        return p_v_given_h, torch.bernoulli(p_v_given_h)

    def train(self, v0, vk, ph0, phk):
        self.W += (torch.mm(v0.t(), ph0) - torch.mm(vk.t(), phk)).t()
        self.b += torch.sum((v0 - vk), 0)
        self.a += torch.sum((ph0 - phk), 0)

nv = len(training_set[0])
nh = 100
batch_size = 100
rbm = RBM(nv, nh)

# Entrenamiento de la RBM

In [ ]:
nb_epoch = 10
for epoch in range(1, nb_epoch + 1):
    train_loss = 0
    s = 0.
    for id_user in range(0, nb_users - batch_size, batch_size):
        vk = training_set[id_user:id_user+batch_size]
        v0 = training_set[id_user:id_user+batch_size]
        ph0,_ = rbm.sample_h(v0)
        for k in range(10):
            _,hk = rbm.sample_h(vk)
            _,vk = rbm.sample_v(hk)
            vk[v0<0] = v0[v0<0]
        phk,_ = rbm.sample_h(vk)
        rbm.train(v0, vk, ph0, phk)
        train_loss += torch.mean(torch.abs(v0[v0>=0] - vk[v0>=0]))
        s += 1.
    print('epoch: '+str(epoch)+' loss: '+str(train_loss/s))


# Testing de la RBM

In [ ]:
test_loss = 0
s = 0.
for id_user in range(nb_users):
    v = training_set[id_user:id_user+1]
    vt = test_set[id_user:id_user+1]
    if len(vt[vt>=0]) > 0:
        _,h = rbm.sample_h(v)
        _,v = rbm.sample_v(h)
        test_loss += torch.mean(torch.abs(vt[vt>=0] - v[vt>=0]))
        s += 1.
print('test loss: '+str(test_loss/s))